In [8]:
import os
import pyaudio
import keyboard
from google.cloud import speech


In [ ]:

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "pazarlik-robotu-04e4fff35c71.json"  # senin yolun


In [10]:

client = speech.SpeechClient()
RATE = 16000
CHUNK = int(RATE / 10)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=RATE,
    language_code="tr-TR"
)

streaming_config = speech.StreamingRecognitionConfig(
    config=config,
    interim_results=False
)

def listen_once():
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=RATE, input=True, frames_per_buffer=CHUNK)

    print("🎙️ Dinliyor (konuşun)...")
    def generator():
        while keyboard.is_pressed('space'):
            data = stream.read(CHUNK, exception_on_overflow=False)
            yield speech.StreamingRecognizeRequest(audio_content=data)
    
    try:
        requests = generator()
        responses = client.streaming_recognize(config=streaming_config, requests=requests)

        for response in responses:
            for result in response.results:
                if result.is_final:
                    print("📝 Tanınan:", result.alternatives[0].transcript)
                    return
    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()

print("🔲 SPACE tuşuna basılı tutarak konuşabilirsiniz (çıkmak için ESC)...")
while True:
    if keyboard.is_pressed("space"):
        listen_once()
    elif keyboard.is_pressed("esc"):
        print("🛑 Çıkılıyor...")
        break



DefaultCredentialsError: File C:/Users/teoman/Desktop/speech_key.json was not found.